# Advent of Code 2022 - Day 16

In [332]:
import re
import networkx as nx
import functools
from collections import defaultdict

In [333]:
test_input = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

In [334]:
inp = open('input.txt','r').read()

In [335]:
d = [[x[6:8],int(re.findall('-?\d+\.?\d*',x)[0]),[y.replace(",","") for y in x.split(" ")[9:]]] for x in inp.splitlines()]
d

[['GS', 0, ['KB', 'GW']],
 ['CB', 0, ['GW', 'CT']],
 ['TP', 0, ['LR', 'TH']],
 ['FI', 3, ['DA', 'AY', 'ZO', 'MP', 'XP']],
 ['WV', 0, ['TH', 'HG']],
 ['EA', 16, ['PL', 'NG', 'AX']],
 ['AT', 9, ['ZO', 'EM']],
 ['WS', 0, ['GW', 'RD']],
 ['MP', 0, ['AA', 'FI']],
 ['GE', 0, ['AX', 'QN']],
 ['SA', 10, ['NI', 'OM', 'RD', 'RC', 'GO']],
 ['NI', 0, ['SA', 'YG']],
 ['GO', 0, ['TH', 'SA']],
 ['IT', 0, ['WB', 'KB']],
 ['NG', 0, ['EA', 'KF']],
 ['RD', 0, ['SA', 'WS']],
 ['LR', 12, ['TP', 'XR']],
 ['TO', 22, ['VW']],
 ['WF', 0, ['XX', 'OO']],
 ['YD', 21, ['NR']],
 ['XR', 0, ['LR', 'KB']],
 ['KF', 0, ['GW', 'NG']],
 ['OO', 0, ['UD', 'WF']],
 ['HG', 0, ['WV', 'YG']],
 ['CT', 0, ['YG', 'CB']],
 ['DA', 0, ['TH', 'FI']],
 ['YY', 0, ['AA', 'YG']],
 ['VW', 0, ['TO', 'EM']],
 ['RC', 0, ['AA', 'SA']],
 ['PL', 0, ['AA', 'EA']],
 ['TH', 14, ['GO', 'WV', 'GJ', 'DA', 'TP']],
 ['QN', 24, ['LC', 'GE']],
 ['XE', 0, ['NA', 'XX']],
 ['XP', 0, ['FI', 'OM']],
 ['AX', 0, ['GE', 'EA']],
 ['EM', 0, ['AT', 'VW']],
 ['NR', 0

In [336]:
rates = {a:b for a,b,c in d}
rates

{'GS': 0,
 'CB': 0,
 'TP': 0,
 'FI': 3,
 'WV': 0,
 'EA': 16,
 'AT': 9,
 'WS': 0,
 'MP': 0,
 'GE': 0,
 'SA': 10,
 'NI': 0,
 'GO': 0,
 'IT': 0,
 'NG': 0,
 'RD': 0,
 'LR': 12,
 'TO': 22,
 'WF': 0,
 'YD': 21,
 'XR': 0,
 'KF': 0,
 'OO': 0,
 'HG': 0,
 'CT': 0,
 'DA': 0,
 'YY': 0,
 'VW': 0,
 'RC': 0,
 'PL': 0,
 'TH': 14,
 'QN': 24,
 'XE': 0,
 'XP': 0,
 'AX': 0,
 'EM': 0,
 'NR': 0,
 'YG': 4,
 'PM': 0,
 'AY': 0,
 'GJ': 0,
 'LC': 0,
 'UD': 17,
 'AA': 0,
 'OM': 0,
 'WB': 0,
 'GW': 11,
 'NA': 7,
 'XX': 20,
 'ZO': 0,
 'KB': 8}

In [337]:
# Create an empty graph
G = nx.Graph()

# Add edges to the graph with weights as number of steps 
for v in d:
    valve, rate, connected = v
    for c_v in connected:
        G.add_edge(valve, c_v)

In [338]:
# create time to travel/open lookup

dist_dict = defaultdict(dict)
for v in set(rates):
    for c in set(rates)-{v}:
        dist_dict[v][c] = nx.shortest_path_length(G,v,c) + 1

In [339]:
options = frozenset([a for a in rates if rates[a]!=0])

In [340]:
@functools.lru_cache(maxsize=None)

def best_choice(current, options, mins_left):
    
    #for a given position and remaining valves, return the highest score possible
    best_score = []
    
    # loop through available options (unopened valves)
    for dest in options:
        
        #if there's enough time to get there and open then we calculate the score
        if dist_dict[current][dest] < mins_left:
            
            new_mins_left = mins_left - (dist_dict[current][dest])
                
            new_sum = (rates[dest] * (new_mins_left))
            
            #valve closed
            new_options = options - {dest}
            
            #calculate the score of subsequent choices recursively
            
            best_score.append((new_mins_left * rates[dest]) + best_choice(dest, new_options, new_mins_left))

    if best_score:
        return max(best_score)
    else: 
        return 0

In [341]:
best_choice("AA",options,30)

1775

In [288]:
## Part 2